<a href="https://colab.research.google.com/github/Rishabh672003/Learning-machine-learning/blob/main/jupyter-notebooks/fetal_health.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting the data and pre-processing it

In [152]:
!pip install -q wfdb

In [153]:
![ ! -f ctu-chb-intrapartum-cardiotocography-database-1.0.0.zi ] && curl -LO https://physionet.org/static/published-projects/ctu-uhb-ctgdb/ctu-chb-intrapartum-cardiotocography-database-1.0.0.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 38.2M  100 38.2M    0     0   675k      0  0:00:58  0:00:58 --:--:--  669k


In [84]:
!unzip -f /content/ctu-chb-intrapartum-cardiotocography-database-1.0.0.zip
!rm /content/ctu-chb-intrapartum-cardiotocography-database-1.0.0.zip

Archive:  /content/ctu-chb-intrapartum-cardiotocography-database-1.0.0.zip
replace ctu-chb-intrapartum-cardiotocography-database-1.0.0/1296.dat? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [85]:
!ls -la /content/__MACOSX/database/signals

ls: cannot access '/content/__MACOSX/database/signals': No such file or directory


In [86]:
!git clone https://github.com/fabiom91/CTU-CHB_Physionet.org.git

fatal: destination path 'CTU-CHB_Physionet.org' already exists and is not an empty directory.


In [87]:
!mkdir /content/CTU-CHB_Physionet.org/physionet_data
!mkdir -p signals
!cp -r /content/ctu-chb-intrapartum-cardiotocography-database-1.0.0/* /content/CTU-CHB_Physionet.org/physionet_data

mkdir: cannot create directory ‘/content/CTU-CHB_Physionet.org/physionet_data’: File exists


In [88]:
import pandas as pd
import wfdb
from os import listdir
from tqdm import tqdm
import numpy as np

def get_all_records():
    rec_list = []
    for file in listdir("/content/CTU-CHB_Physionet.org/physionet_data"):
        rec = file[:file.find('.')]
        try:
            rec = int(rec)
            rec_list.append(rec)
        except:
            pass
    rec_list = [str(i) for i in rec_list]
    return rec_list

def create_signals_database(rec):
    sample = wfdb.rdsamp("/content/CTU-CHB_Physionet.org/physionet_data/%s" % rec)
    df = pd.DataFrame(sample[0], columns=['FHR','UC'])
    df.index.name = 'seconds'
    df.to_csv('signals/%s.csv' % rec)

def create_ann_dataframe(rec):
    sample = wfdb.rdsamp("/content/CTU-CHB_Physionet.org/physionet_data/%s" % rec)
    ann = sample[1]['comments'][1:]
    ann_dic = {}
    for a in ann:
        if '--' in a:
            ann.remove(a)

    for a in ann:
        key = a[:a.find('  ')]
        if a.find('  ') == -1:
            key = a[:a.find(' ')]
        inv = a[::-1]
        value = inv[:inv.find(' ')][::-1]
        value = float(value)
        ann_dic[key] = [value]

    df1 = pd.DataFrame.from_dict(ann_dic).T
    df1 = df1.rename(columns={0:rec})
    return df1

def append_ann_dataframes(df,df1):
    rec = df1.columns[0]
    df[rec] = df1[rec]
    return df


df = pd.DataFrame()
for rec in tqdm(get_all_records()):
    create_signals_database(rec)
    df = append_ann_dataframes(df,create_ann_dataframe(rec))
df.to_csv('ann_db.csv')

print('DONE!')

 10%|▉         | 107/1104 [00:03<00:59, 16.72it/s]<ipython-input-88-241d791dcf73>:48: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[rec] = df1[rec]
<ipython-input-88-241d791dcf73>:48: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[rec] = df1[rec]
<ipython-input-88-241d791dcf73>:48: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame,

DONE!


# Starting the work on dataset

In [125]:
import pandas as pd

In [126]:
df = pd.read_csv("/content/ann_db.csv")

In [127]:
df.head()

,Unnamed: 0,1365,2035,1204,1073,1217,1176,1441,1108,1474,...,1093,1349,1306,1232,1326,2017,1224,1211,1355,1167
0,pH,7.39,7.26,7.32,7.13,7.18,7.22,7.23,7.02,7.27,...,7.27,7.32,7.39,7.23,7.29,7.32,7.33,6.98,7.27,7.36
1,BDecf,3.32,2.23,3.68,7.11,4.57,7.80,5.27,12.81,3.43,...,0.01,0.89,4.14,3.00,1.84,-0.16,2.72,NaN,4.06,5.03
2,pCO2,4.60,7.20,5.60,8.40,8.20,6.10,6.80,8.50,6.60,...,7.70,6.40,4.40,7.60,6.70,6.70,5.70,NaN,6.40,4.60
3,BE,-4.50,-4.30,-4.90,-9.80,-6.70,-9.60,-6.90,-15.50,-5.30,...,-2.20,-1.80,-4.10,-5.40,-3.80,-2.00,-4.10,NaN,-5.60,-5.70
4,Apgar1,9.00,8.00,9.00,10.00,7.00,8.00,9.00,7.00,10.00,...,9.00,9.00,10.00,8.00,9.00,10.00,9.00,6.00,9.00,9.00


In [128]:
df.shape

(35, 553)

In [129]:
df.head(20)

,Unnamed: 0,1365,2035,1204,1073,1217,1176,1441,1108,1474,...,1093,1349,1306,1232,1326,2017,1224,1211,1355,1167
0,pH,7.39,7.26,7.32,7.13,7.18,7.22,7.23,7.02,7.27,...,7.27,7.32,7.39,7.23,7.29,7.32,7.33,6.98,7.27,7.36
1,BDecf,3.32,2.23,3.68,7.11,4.57,7.80,5.27,12.81,3.43,...,0.01,0.89,4.14,3.00,1.84,-0.16,2.72,NaN,4.06,5.03
2,pCO2,4.60,7.20,5.60,8.40,8.20,6.10,6.80,8.50,6.60,...,7.70,6.40,4.40,7.60,6.70,6.70,5.70,NaN,6.40,4.60
3,BE,-4.50,-4.30,-4.90,-9.80,-6.70,-9.60,-6.90,-15.50,-5.30,...,-2.20,-1.80,-4.10,-5.40,-3.80,-2.00,-4.10,NaN,-5.60,-5.70
4,Apgar1,9.00,8.00,9.00,10.00,7.00,8.00,9.00,7.00,10.00,...,9.00,9.00,10.00,8.00,9.00,10.00,9.00,6.00,9.00,9.00
5,Apgar5,10.00,9.00,9.00,10.00,8.00,9.00,10.00,8.00,10.00,...,10.00,10.00,10.00,8.00,9.00,10.00,9.00,8.00,10.00,10.00
6,NICU days,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,Seizures,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,HIE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,Intubation,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [130]:
import numpy as np

new_cols = pd.DataFrame(columns=['index','FHR_mean','UC_mean', "FHR_rms", "UC_rms", "FHR_max", "UC_max"])
for i in range(1001, 2046):
    f = f"signals/{i}.csv"
    try:
      df_temp = pd.read_csv(f)

      fhr_mean, uc_mean = df_temp["FHR"].mean(), df_temp["UC"].mean()
      fhr_rms = np.sqrt(np.mean(np.square(df_temp['FHR'])))
      uc_rms = np.sqrt(np.mean(np.square(df_temp['UC'])))
      fhr_max, uc_max = df_temp.loc[df_temp['FHR'].idxmax()]["FHR"], df_temp.loc[df_temp['UC'].idxmax()]["UC"]
      new_cols.loc[0] = [i-1001, fhr_mean, uc_mean, fhr_rms, uc_rms, fhr_max, uc_max]
      new_cols.index += 1
      new_cols = new_cols.sort_index()
    except:
      continue


In [131]:
new_cols.head()

,index,FHR_mean,UC_mean,FHR_rms,UC_rms,FHR_max,UC_max
1,1044.0,96.125571,14.546898,106.547922,22.618719,167.50,127.0
2,1043.0,115.084445,15.543302,133.228479,22.951402,182.00,102.5
3,1042.0,82.792328,18.059265,102.957196,28.069936,233.50,127.0
4,1041.0,90.791597,39.765486,106.856817,48.074035,160.25,96.5
5,1040.0,105.066601,16.282025,127.023054,23.284595,178.00,98.0


In [132]:
df.columns

Index(['Unnamed: 0', '1365', '2035', '1204', '1073', '1217', '1176', '1441',
       '1108', '1474',
       ...
       '1093', '1349', '1306', '1232', '1326', '2017', '1224', '1211', '1355',
       '1167'],
      dtype='object', length=553)

In [133]:
df = df.T

In [134]:
df = df.reset_index(drop=True)

In [145]:
df.columns = [x for x in df.loc[0]]
df = df[1:].reset_index(drop=True)

In [146]:
df.head()

,0,pH,BDecf,pCO2,BE,Apgar1,Apgar5,NICU days,Seizures,HIE,...,Induced,I.stage,NoProgress,CK/KP,II.stage,Deliv. type,dbID,Rec. type,Pos. II.st.,Sig2Birth
0,1,7.39,3.32,4.6,-4.5,9.0,10.0,0.0,0.0,0.0,...,0.0,410.0,0.0,0.0,10.0,1.0,1073381.0,1.0,14400.0,0.0
1,2,7.26,2.23,7.2,-4.3,8.0,9.0,0.0,0.0,0.0,...,0.0,205.0,1.0,0.0,-1.0,2.0,1062347.0,12.0,-1.0,0.0
2,3,7.32,3.68,5.6,-4.9,9.0,9.0,0.0,0.0,0.0,...,0.0,290.0,0.0,0.0,10.0,1.0,1048756.0,2.0,14400.0,0.0
3,4,7.13,7.11,8.4,-9.8,10.0,10.0,0.0,0.0,0.0,...,0.0,107.0,0.0,0.0,30.0,1.0,1014391.0,1.0,14400.0,0.0
4,5,7.18,4.57,8.2,-6.7,7.0,8.0,0.0,0.0,0.0,...,0.0,241.0,0.0,0.0,10.0,1.0,1051153.0,2.0,14400.0,0.0


In [147]:
df = df.reset_index()  # Reset index to make it a column
new_cols = new_cols.rename(columns={'index': 'index'})

# Merge on 'index'
merged_df = df.merge(new_cols, on='index', how='inner')

merged_df.head()

,index,0,pH,BDecf,pCO2,BE,Apgar1,Apgar5,NICU days,Seizures,...,dbID,Rec. type,Pos. II.st.,Sig2Birth,FHR_mean,UC_mean,FHR_rms,UC_rms,FHR_max,UC_max
0,0,1,7.39,3.32,4.6,-4.5,9.0,10.0,0.0,0.0,...,1073381.0,1.0,14400.0,0.0,106.984336,24.815130,122.357020,33.143752,193.00,127.0
1,1,2,7.26,2.23,7.2,-4.3,8.0,9.0,0.0,0.0,...,1062347.0,12.0,-1.0,0.0,122.064076,24.777604,135.710988,34.975993,224.00,127.0
2,2,3,7.32,3.68,5.6,-4.9,9.0,9.0,0.0,0.0,...,1048756.0,2.0,14400.0,0.0,97.129514,14.892778,109.963878,26.323443,174.50,100.0
3,3,4,7.13,7.11,8.4,-9.8,10.0,10.0,0.0,0.0,...,1014391.0,1.0,14400.0,0.0,133.030074,22.084762,135.560556,37.570082,166.75,100.0
4,4,5,7.18,4.57,8.2,-6.7,7.0,8.0,0.0,0.0,...,1051153.0,2.0,14400.0,0.0,92.547375,15.335806,105.951334,22.315540,229.00,123.0


In [151]:
merged_df.drop(["index", 0], axis=1)

,pH,BDecf,pCO2,BE,Apgar1,Apgar5,NICU days,Seizures,HIE,Intubation,...,dbID,Rec. type,Pos. II.st.,Sig2Birth,FHR_mean,UC_mean,FHR_rms,UC_rms,FHR_max,UC_max
0,7.39,3.32,4.6,-4.5,9.0,10.0,0.0,0.0,0.0,0.0,...,1073381.0,1.0,14400.0,0.0,106.984336,24.815130,122.357020,33.143752,193.00,127.0
1,7.26,2.23,7.2,-4.3,8.0,9.0,0.0,0.0,0.0,0.0,...,1062347.0,12.0,-1.0,0.0,122.064076,24.777604,135.710988,34.975993,224.00,127.0
2,7.32,3.68,5.6,-4.9,9.0,9.0,0.0,0.0,0.0,0.0,...,1048756.0,2.0,14400.0,0.0,97.129514,14.892778,109.963878,26.323443,174.50,100.0
3,7.13,7.11,8.4,-9.8,10.0,10.0,0.0,0.0,0.0,0.0,...,1014391.0,1.0,14400.0,0.0,133.030074,22.084762,135.560556,37.570082,166.75,100.0
4,7.18,4.57,8.2,-6.7,7.0,8.0,0.0,0.0,0.0,0.0,...,1051153.0,2.0,14400.0,0.0,92.547375,15.335806,105.951334,22.315540,229.00,123.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,7.25,0.79,7.9,-2.3,10.0,10.0,0.0,0.0,0.0,0.0,...,1057301.0,1.0,14400.0,0.0,101.962123,28.217560,110.624265,32.176172,226.25,127.0
502,7.27,4.68,6.2,-6.1,8.0,9.0,0.0,0.0,0.0,0.0,...,1020849.0,1.0,14400.0,0.0,117.663986,7.808306,122.720699,13.644553,164.00,93.0
503,7.1,8.61,8.5,-11.5,9.0,10.0,0.0,0.0,0.0,0.0,...,1113342.0,1.0,14400.0,0.0,105.995030,25.672768,121.027884,38.961869,208.00,127.0
504,7.16,1.93,9.8,-4.8,9.0,9.0,0.0,0.0,0.0,0.0,...,1116608.0,1.0,14400.0,0.0,127.865069,11.691444,129.991798,20.086897,173.50,100.0
